# Propane Tutorial


### Imports

In [1]:
import itertools
import string

import hoomd
import hoomd.md
from hoomd.deprecated.init import read_xml
import numpy as np

from msibi import MSIBI, State, Pair, mie

### Remove files generated during CG simulation

In [2]:
%%bash
rm -rf rdfs/pair* f_fits.log state*/*.txt state*/run.py state*/*query.dcd

In [3]:
kTs = [0.5, 1.5, 2.0]

In [ ]:
# This cell takes 40 minutes. Don't uncomment and rerun it unless you have a good reason.
# trajectory{kT}.gsd files were created here
# lj params from doi:10.1088/0957-4484/18/42/424007
#         ε(kcal/mol)  σ(A ̊)
# Propane    0.553     4.66
for i,kT in enumerate(kTs):
    hoomd.context.initialize("")
    system = read_xml(f"state_{i}/start.hoomdxml")
    
    nl = hoomd.md.nlist.cell()
    lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)
    lj.pair_coeff.set("C3", "C3", epsilon=1.0, sigma=1.0)
    hoomd.md.integrate.mode_standard(dt=0.001)
    _all = hoomd.group.all()
    nvt = hoomd.md.integrate.nvt(group=_all, kT=kT, tau=1)
    nvt.randomize_velocities(seed=23)
    hoomd.analyze.log(
        filename=f"state_{i}/propane_kT{kT}.log",
        quantities=["time", "temperature", "potential_energy"],
        period=100,
        overwrite=True
    )
    hoomd.dump.gsd(f"state_{i}/trajectory{kT}.gsd", period=5e3, group=_all, overwrite=True)
    hoomd.run(1e6)

HOOMD-blue 2.9.6 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 03/17/2021
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the CPU
notice(2): Reading state_0/start.hoomdxml...
notice(2): No version specified in hoomd_xml root node: assuming 1.0
notice(2): --- hoomd_xml file read summary
notice(2): 1024 positions at timestep 0
notice(2): 1024 masses
notice(2): 1 particle types
notice(2): 1024 charges
notice(2): Group "all" created containing 1024 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 1024
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
fig.suptitle("Potential Energy vs Timestep")
for i, kT in enumerate(kTs):   
    log = np.loadtxt(f"state_{i}/propane_kT{kT}.log", skiprows=1)
    ax1.plot(log[:,0],log[:,3], label=f"kT={kT}")
    ax2.plot(log[-50:,0],log[-50:,3], label=f"kT={kT}")
    ax1.set_xlabel("Timestep")
    ax2.set_xlabel("Timestep")
    ax1.set_ylabel("Potential Energy (AU)")
    ax1.legend()
    ax2.legend()

### Set up global parameters
Cutoff radius set to 5.0 units. Parameters including number of data points and potential cutoff are passed to `MSIBI`.

### Specify states
States each are initialized with different temperatures, directories, and start.hoomdxml files. 

A list `states` contains all the individual states: `stateA`, `stateB`, `stateC`.

In [ ]:
state0 = State(
    kT=0.5, 
    state_dir="./state_0", 
    traj_file="trajectory0.5.gsd",
    name="state0", 
    backup_trajectory=True
)
state1 = State(
    kT=1.5, 
    state_dir="./state_1", 
    traj_file="trajectory1.5.gsd",
    name="state1",
    backup_trajectory=True
)
state2 = State(
    kT=2.0, 
    state_dir="./state_2", 
    traj_file="trajectory2.0.gsd",
    name="state2", 
    backup_trajectory=True
)
states = [state0, state1, state2]

### Specify pairs

Creates a list of all the possible indices for the 1024 atoms. 

Passes the type of interaction to be optimized, a `C3` to itself, to `Pair`. Sets the alpha values to `1.0`

In [ ]:
indices = list(itertools.combinations(range(1024), 2))  # all-all for 1024 atoms

initial_guess = mie(opt.pot_r, 1.0, 1.0)  # 1-D array of potential values.
alphabet = ["A", "B", "C"]
rdf_targets = [np.loadtxt(f"rdfs/C3-C3-state_{i}.txt") for i in alphabet]

pair0 = Pair("C3", "C3", initial_guess)
alphas = [1.0, 1.0, 1.0]

### Add targets to pair

Loops through each `state`, `target`, and `alpha` in `zip`. Adds the appropriate states, and converts `pair0` into a list for the `optimize()` function.

In [ ]:
for state, target, alpha in zip(states, rdf_targets, alphas):
    pair0.add_state(state, target, alpha, indices)
pairs = [pair0]

### Do magic

Sprinkle fairy dust over the code.


Calls the `optimize` function with the parameters given. 
Performs five iterations, with each successive iteration usually producing finer, better output. 
Uses the `hoomd` engine to run the simulations.

In [ ]:
opt.optimize(states, pairs, n_iterations=5, engine="hoomd")